In [500]:
import pandas as pd
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## Loading the new dataset

In [501]:
df = pd.read_excel("../data/processed/loaded_data.xlsx")

## We will pre-process the data and see what are all the missing and duplicate values present in the dataset

### Handling Missing values

In [502]:
df.isnull().sum()

id                               17
acceptance_rate                7923
institution                      25
program                        1242
degree_type                    1177
decision                       1180
undergrad_gpa                 62618
gre_quantitative_reasoning    96604
gre_verbal_reasoning          97836
analytical_writing            98529
notes                         45900
gre_total                     97998
dtype: int64

### Columns that should NOT be missing (ideally):
id
→ Must be present and unique for every record.

institution
→ The university/school applied to — essential info.
🔸 Should not be missing.

program
→ Program applied to (e.g., MS CS) — needed for analysis.
🔸 Should not be missing.

degree_type
→ Such as MS, PhD, etc. — core info.
🔸 Should not be missing.

decision
→ Admission result (Accepted/Rejected) — central to analysis.
🔸 Should not be missing.



In [503]:
df = df.dropna(subset=['acceptance_rate', 'institution', 'program', 'degree_type', 'decision'])

In [505]:
df.shape

(100963, 12)

In [506]:
# Step 1: Find duplicate IDs
duplicate_ids = df[df.duplicated('id', keep=False)]

In [507]:
# Convert ID to string to ensure consistent type
duplicate_ids.loc[:,'id'] = duplicate_ids['id'].astype(str)

# Step 2: Sort by 'id' to group duplicates together
duplicate_ids_sorted = duplicate_ids.sort_values(by='id')


In [508]:
# Step 3: Inspect all rows with the same id
# You can group them and print each group for manual comparison
for dup_id, group in duplicate_ids_sorted.groupby('id'):
    if len(group) > 1:
        print(f"\nDuplicate ID: {dup_id}")
        print(group)


Duplicate ID: 817731
           id acceptance_rate                      institution  \
83003  817731             44%  Georgia Institute Of Technology   
82983  817731             44%  Georgia Institute Of Technology   

                program degree_type  decision undergrad_gpa  \
83003  Computer Science     Masters  Rejected           3.6   
82983  Computer Science     Masters  Rejected           3.6   

      gre_quantitative_reasoning gre_verbal_reasoning analytical_writing  \
83003                        165                  159                  4   
82983                        165                  159                  4   

      notes gre_total  
83003   NaN       324  
82983   NaN       324  

Duplicate ID: 817732
           id acceptance_rate        institution           program  \
82982  817732             45%  Purdue University  Computer Science   
83002  817732             45%  Purdue University  Computer Science   

      degree_type  decision undergrad_gpa gre_quantitat

In [509]:
# Remove rows that are exact duplicates (all columns identical)
df = df.drop_duplicates()

In [510]:
df.describe()

id acceptance_rate               institution           program  \
count   100629          100629                    100629            100629   
unique  100626             172                      1130              7910   
top     964029             40%  University of California  Computer Science   
freq         2            8486                      4081              7527   

       degree_type  decision  undergrad_gpa  gre_quantitative_reasoning  \
count       100629    100629          44646                       12256   
unique           9         9            348                         143   
top            PhD  Accepted              4                         170   
freq         64772     37248           5320                        2106   

        gre_verbal_reasoning  analytical_writing                  notes  \
count                  11095               10435                  58553   
unique                    86                  98                  54280   
top                      162                   4  Email to check portal   
freq                     793                2978                    264   

        gre_total  
count       10940  
unique        185  
top           330  
freq          485

In [511]:
print(df['id'].duplicated().sum())

3


In [512]:
duplicate_ids = df[df.duplicated('id', keep=False)]


In [513]:
df = df.drop_duplicates(subset='id', keep='first')

In [514]:
print(df['id'].duplicated().sum())

0


In [515]:
df = df[df['gre_verbal_reasoning'] != 'GRE Verbal']

In [516]:
df.acceptance_rate.isna().sum()

0

In [517]:
df.describe()

id acceptance_rate               institution           program  \
count   100625          100625                    100625            100625   
unique  100625             171                      1129              7909   
top     879550             40%  University of California  Computer Science   
freq         1            8486                      4080              7526   

       degree_type  decision  undergrad_gpa  gre_quantitative_reasoning  \
count       100625    100625          44642                       12255   
unique           8         8            347                         142   
top            PhD  Accepted              4                         170   
freq         64771     37245           5320                        2106   

        gre_verbal_reasoning  analytical_writing                  notes  \
count                  11094               10434                  58552   
unique                    85                  97                  54279   
top                      162                   4  Email to check portal   
freq                     793                2978                    264   

        gre_total  
count       10939  
unique        184  
top           330  
freq          485

In [518]:
df.to_excel("../data/processed/preprocessed_data.xlsx", index=False)